In [60]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.join(os.path.abspath(''), '../'))

from models.graph_sage_model import GraphSAGE
from pprint import pprint
from dataset.data_loader import DataLoader, playtime_forever_users_games_edge_scoring_function

import networkx as nx
from torch_geometric.data import Data
import torch
from torch_geometric.nn import SAGEConv
import pickle
from utils.utils import get_game_name_and_scores

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
%reload_ext autoreload

In [3]:
data_loader = DataLoader(num_users_to_load_per_snowball=100, users_games_edge_scoring_function=playtime_forever_users_games_edge_scoring_function,
    user_game_edge_embeddings=['playtime_forever'],
    cache_local_dataset=True, game_embeddings=['name'])
data_loader.load_random_edge_train_test_split(train_percentage=0.9, test_percentage=0.1, seed=0)


In [63]:
model = GraphSAGE(
    hidden_channels=[32, 64],  
    out_channels=16,
)
model.set_data_loader(data_loader)
model.new_seed(123)

In [83]:
model.train(debug=False)

Training:  42%|████▏     | 42/100 [00:05<00:07,  7.40it/s]

Stopping early at epoch 43 epochs with minimal loss improvement.


In [49]:
# # Assuming 'data' is your PyTorch Geometric Data object
# score = model.get_score_between_user_and_game(730, 76561199045179546, data)
# print(score)

In [84]:
display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(76561198835352289)[:20]))

,id,name,score
0,655360,Aftermath Y2K,0.140033
1,655380,Dig Dog,0.130901
2,2097180,OxU,0.130765
3,786470,Border closure 边境封闭,0.130421
4,655370,Train Bandit,0.130105
5,262190,Zombeer,0.130078
6,1966130,Strategic Command: American Civil War,0.129564
7,655420,Simple RTS,0.129050
8,524350,TileDynasty FPS Arena,0.128879
9,655430,Tiger Hunt,0.128365
